In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import r2_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model

%matplotlib inline

In [2]:
infile_old = '../data/BEA-RegionalIncomeByIndustry/CA5_1969_2000_MSA.csv'
infile = '../data/BEA-RegionalIncomeByIndustry/CA5N_2001_2013_MSA.csv'

df_old = pd.read_csv(infile_old)
df = pd.read_csv(infile)

df_old


/usr/local/lib/python2.7/dist-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


,GeoFIPS,GeoName,Region,Table,LineCode,IndustryClassification,Description,1969,1970,1971,...,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000
0,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,10,...,Personal income,395971,443561,474234,...,2376433,2538036,2663665,2722081,2918059,3109606,3329032,3490073,3639311,3741391
1,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,20,...,Population (persons) 2/,125138,122505,124841,...,147400,149917,152909,153779,156097,156351,157405,158264,159755,160288
2,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,30,...,Per capita personal income (dollars),3164,3621,3799,...,16122,16930,17420,17701,18694,19889,21149,22052,22781,23342
3,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,35,...,Earnings by place of work,314729,345979,368356,...,1640392,1759145,1834506,1895221,1995628,2131048,2326369,2432863,2579609,2598895
4,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,36,...,less: Contributions for government social in...,18303,19934,22303,...,168262,176727,184213,193361,203789,214107,227052,232301,240171,244306
5,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,37,...,Employee and self-employed contributions f...,9557,10240,11245,...,87427,91460,95847,101582,108123,113405,120785,123957,128823,129272
6,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,38,...,Employer contributions for government soci...,8746,9694,11058,...,80835,85267,88366,91779,95666,100702,106267,108344,111348,115034
7,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,42,...,plus: Adjustment for residence 4/,1495,1863,2003,...,5066,6192,6245,4630,3561,3165,4426,6403,7632,10171
8,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,45,...,equals: Net earnings by place of residence,297921,327908,348056,...,1477196,1588610,1656538,1706490,1795400,1920106,2103743,2206965,2347070,2364760
9,10180,"Abilene, TX (Metropolitan Statistical Area)",NaN,CA5,46,...,"plus: Dividends, interest, and rent 5/",61986,73938,78915,...,542650,542405,579228,557362,624395,657686,668241,720342,709218,772092


In [3]:
# clean the data columns
import re

data_df = df[[str(x) for x in range(2001,2014)]]


def clean_data_entry(x):
    if isinstance(x, basestring):
        m = re.match("\d+",x)
        if m:
            return float(m.group())
        else:
            return np.nan
    else:
        return x

clean_df = df.copy()
for yr in range(2001,2014):
    clean_df[str(yr)] = clean_df[str(yr)].apply(clean_data_entry)

print clean_df.head(3)

  GeoFIPS                                      GeoName  Region Table  \
0   10180  Abilene, TX (Metropolitan Statistical Area)     NaN  CA5N   
1   10180  Abilene, TX (Metropolitan Statistical Area)     NaN  CA5N   
2   10180  Abilene, TX (Metropolitan Statistical Area)     NaN  CA5N   

   LineCode IndustryClassification                           Description  \
0        10                    ...                       Personal income   
1        20                    ...               Population (persons) 2/   
2        30                    ...  Per capita personal income (dollars)   

      2001     2002     2003     2004     2005     2006     2007     2008  \
0  3826121  3957936  4181505  4363728  4588171  4863397  5211004  5811434   
1   158917   159012   158810   160156   160761   161389   162023   162508   
2    24076    24891    26330    27247    28540    30135    32162    35761   

      2009     2010     2011     2012     2013  
0  5465772  5685546  6111898  6546641  6743442  

In [4]:
for geo in clean_df['GeoFIPS'].unique():
    numna = clean_df[clean_df['GeoFIPS']==geo][[str(x) for x in range(2001,2014)]].isnull().sum().sum()
    print numna, geo

435 10180
245 10420
606 10500
294 10540
505 10580
362 10740
381 10780
451 10900
322 11020
506 11100
401 11180
354 11260
288 11460
339 11500
393 11540
525 11700
604 12020
518 12060
331 12100
299 12220
624 12260
313 12420
166 12540
380 12580
308 12620
255 12700
506 12940
365 12980
440 13020
438 13140
527 13220
242 13380
304 13460
373 13740
427 13780
387 13820
584 13900
903 13980
581 14010
511 14020
631 14100
370 14260
283 14460
204 14500
557 14540
286 14740
195 14860
199 15180
539 15260
193 15380
291 15500
419 15540
458 15680
259 15940
196 15980
553 16020
613 16060
432 16180
378 16220
390 16300
317 16540
500 16580
434 16620
417 16700
434 16740
779 16820
467 16860
378 16940
310 16980
226 17020
460 17140
650 17300
496 17420
319 17460
222 17660
463 17780
254 17820
348 17860
518 17900
595 17980
374 18020
452 18140
265 18580
428 18700
392 18880
592 19060
283 19100
553 19140
398 19180
304 19300
523 19340
404 19380
537 19460
424 19500
337 19660
399 19740
419 19780
309 19820
529 20020
811 20100


In [5]:
one_df = clean_df[clean_df['GeoFIPS']==10900]

one_df[[str(x) for x in range(2001,2014)]].isnull().sum()

2001    37
2002    38
2003    35
2004    33
2005    35
2006    34
2007    32
2008    32
2009    32
2010    33
2011    33
2012    33
2013    44
dtype: int64

In [6]:
# one_yr_df = one_df[['LineCode','IndustryClassification','Description','2004']]

n = 500
print one_yr_df[one_yr_df['LineCode'] == n]
one_yr_df[one_yr_df['LineCode'] > n][one_yr_df['LineCode'] < n+1000]

NameError: name 'one_yr_df' is not defined

In [87]:
print one_yr_df[one_yr_df['LineCode'] < 100]

print 'industries sum:', one_yr_df[one_yr_df['LineCode']%100==0]['2004'].sum()


     LineCode IndustryClassification  \
917        10                    ...   
918        20                    ...   
919        30                    ...   
920        35                    ...   
921        36                    ...   
922        37                    ...   
923        38                    ...   
924        42                    ...   
925        45                    ...   
926        46                    ...   
927        47                    ...   
928        50                    ...   
929        60                    ...   
930        61                    ...   
931        62                    ...   
932        70                    ...   
933        71                    ...   
934        72                    ...   
935        81                111-112   
936        82                    ...   
937        90                113-814   

                                           Description      2004  
917                                    Personal inco